In [5]:
import yfinance as yf
import pandas as pd
from sktime.utils.plotting import plot_series

# from sktime.datasets import load_airline
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.theta import ThetaForecaster
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
import pandas as pd
from sktime.utils.plotting import plot_series
import sys
from sklearn.metrics import mean_absolute_error, r2_score
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.fbprophet import Prophet

import matplotlib.pyplot as plt
import numpy as np

In [2]:
symbol='^NSEI'
ticker = yf.Ticker(symbol)
data=ticker.history(period='2d',interval='1m')
data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2024-04-15 09:15:00+05:30,22356.699219,22369.099609,22319.300781,22358.050781,0,0.0,0.0
2024-04-15 09:16:00+05:30,22353.599609,22363.650391,22340.949219,22343.349609,0,0.0,0.0
2024-04-15 09:17:00+05:30,22341.650391,22344.300781,22325.300781,22329.550781,0,0.0,0.0
2024-04-15 09:18:00+05:30,22329.599609,22334.449219,22308.599609,22315.449219,0,0.0,0.0
2024-04-15 09:19:00+05:30,22314.750000,22330.449219,22313.750000,22325.199219,0,0.0,0.0


In [3]:
df=data.resample(rule='min').sum()
df.drop(columns=['Volume','Dividends','Stock Splits'],inplace=True)
df.head()

,Open,High,Low,Close
Datetime,,,,
2024-04-15 09:15:00+05:30,22356.699219,22369.099609,22319.300781,22358.050781
2024-04-15 09:16:00+05:30,22353.599609,22363.650391,22340.949219,22343.349609
2024-04-15 09:17:00+05:30,22341.650391,22344.300781,22325.300781,22329.550781
2024-04-15 09:18:00+05:30,22329.599609,22334.449219,22308.599609,22315.449219
2024-04-15 09:19:00+05:30,22314.750000,22330.449219,22313.750000,22325.199219


In [6]:
def sktime_forecast(dataset, horizon, forecaster, validation=False, confidence=0.9, frequency="D"):
    """Loop over a time series dataframe, train an sktime forecasting model, and visualize the results.

    Args:
        dataset (pd.DataFrame): Input time series DataFrame with datetime index
        horizon (int): Forecast horizon
        forecaster (sktime.forecasting): Configured forecaster
        validation (bool, optional): . Defaults to False.
        confidence (float, optional): Confidence level. Defaults to 0.9.
        frequency (str, optional): . Defaults to "D".
    """
    
    # Adjust frequency
    forecast_df = dataset.resample(rule=frequency).sum()

    # Interpolate missing periods (if any)
    forecast_df = forecast_df.interpolate(method="time")

    for col in dataset.columns:

        # Use train/test split to validate forecaster
        if validation:
            df = forecast_df[col]

            y_train = df[:-horizon]
            y_test = df.tail(horizon)

            forecaster.fit(y_train)
            fh = ForecastingHorizon(y_test.index, is_relative=False)
            y_pred = forecaster.predict(fh)
            ci = forecaster.predict_interval(fh, coverage=confidence).astype("float")
            y_true = df.tail(horizon)

            mae = mean_absolute_error(y_true, y_pred)

        # Make predictions beyond the dataset
        if not validation:
            df = forecast_df[col].dropna()
            forecaster.fit(df)

            last_date = df.index.max()
            fh = ForecastingHorizon(
                pd.date_range(str(last_date), periods=horizon, freq=frequency),
                is_relative=False,
            )

            y_pred = forecaster.predict(fh)
            ci = forecaster.predict_interval(fh, coverage=confidence).astype("float")
            mae = np.nan
            
            # Visualize results
        plt.plot(
            df.tail(horizon * 3),
            label="Actual",
            color="black",
        )
        plt.gca().fill_between(
            ci.index, (ci.iloc[:, 0]), (ci.iloc[:, 1]), color="b", alpha=0.1
        )
        plt.plot(y_pred, label="Predicted")
        # plt.title(
        #     f"{horizon} day forecast for {col} (mae: {round(mae, 2)}, confidence: {confidence*100}%)"
        # )
        plt.ylim(bottom=0)
        # plt.legend()
        plt.grid(False)
        plt.show()

forecaster = Prophet( weekly_seasonality=True)
sktime_forecast(dataset=df, horizon=120, forecaster=forecaster, validation=False)

/home/rahul/Desktop/dApp/DAPP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


ValueError: Column ds has timezone specified, which is not supported. Remove timezone.